In [81]:
# import cv2
# import numpy as np

# # Load images
# img1 = cv2.imread('../src/ransac1.jpg', cv2.IMREAD_GRAYSCALE)
# img2 = cv2.imread('../src/ransac2.jpg', cv2.IMREAD_GRAYSCALE)
# # Step 1: Feature Detection and Description
# detector = cv2.SIFT_create()
# kp1, des1 = detector.detectAndCompute(img1, None)
# kp2, des2 = detector.detectAndCompute(img2, None)
# # Step 2: Feature Matching
# matcher = cv2.BFMatcher()
# matches = matcher.knnMatch(des1, des2, k=2)
# # Step 3: RANSAC to find inliers
# good_matches = []
# for m, n in matches:
#     if m.distance < 0.75 * n.distance:
#         good_matches.append(m)

# if len(good_matches) > 4:
#     src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
#     dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    
#     # Estimate homography using RANSAC
#     H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
#     matches_mask = mask.ravel().tolist()
# else:
#     raise AssertionError("Can't find enough keypoints.")

# # Step 4: Visualize Correspondences
# draw_params = dict(matchColor=(0, 255, 0),  # draw matches in green color
#                    singlePointColor=None,
#                    matchesMask=matches_mask,  # draw only inliers
#                    flags=cv2.DrawMatchesFlags_DEFAULT)
# correspondence_img = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, **draw_params)
# # Draw outlier matches in red
# outliers_mask = np.logical_not(matches_mask)
# for m in np.array(good_matches)[outliers_mask]:
#     pt1 = tuple(map(int, kp1[m.queryIdx].pt))
#     pt2 = tuple(map(int, kp2[m.trainIdx].pt))
#     cv2.line(correspondence_img, pt1, pt2, (0, 0, 255), 1)
# # Show the result
# cv2.imshow("Correspondences", correspondence_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [82]:
import cv2 
import numpy as np 

# Load images 
img1 = cv2.imread('../src/ransac1.jpg', cv2.IMREAD_GRAYSCALE) 
img2 = cv2.imread('../src/ransac2.jpg', cv2.IMREAD_GRAYSCALE) 

# Step 1: Feature Detection and Description 
detector = cv2.SIFT_create() 
kp1, des1 = detector.detectAndCompute(img1, None) 
kp2, des2 = detector.detectAndCompute(img2, None) 

# Compute correspondences matrix
correspondences = np.zeros((des1.shape[0], des2.shape[0])) 
for i in range(des1.shape[0]): 
    for j in range(des2.shape[0]): 
        diff = np.sum((des1[i]-des2[j])**2) 
        correspondences[i,j] = np.sum((des1[i]-des2[j])**2) 

# Normalize correspondences matrix between 0 and 1
correspondences = (correspondences - np.min(correspondences)) / (np.max(correspondences) - np.min(correspondences)) 
correspondences = np.where(correspondences < .005, 1, 0)

# Draw correspondences
matches = []
for i in range(des1.shape[0]):
    for j in range(des2.shape[0]):
        if correspondences[i, j] == 1:
            matches.append(cv2.DMatch(i, j, 1))

# Draw matches
img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Display the matches
cv2.imshow('Matches', img_matches)
cv2.waitKey(0)
cv2.destroyAllWindows()